<a href="https://colab.research.google.com/github/martinj2-dot/CSC386-Wall-Follower/blob/main/username_CSC386_Wall_Follower.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A00: Wall Follower
* This assignment should be done individually or with a partner.
* You should seek help completing this assignment from the TAs at the evening lab.

### Learning Objectives 
* Practice breaking a larger problem down into smaller pieces using functions.
* Gain practice manipulating arrays.
* Establish and understanding of the Wall-Follower algorithm and its purpose

### How to Start
* To begin, save a copy in Drive and share it with all members of your team. **Colab does not let you make changes until you save a copy**
* Change the file name of this document to username1, username2 - A00: Wall Follower (for example, neillz, martinj2 - A00: Wall Follower). 

### Additional Notes
This assignment utilizes Google Colab. If you are unfamiliar with Colab, don’t worry. The idea is that you can write both code and read text. **Real-time saving was deprecated in 2018. Your partner must refresh the page to see your changes.**

The assignment has three activities: Drive Square, Drive Safe, and Wall Follower. There is no git workflow associated with this project. Once you are finished, submit the link to Moodle.

In [ ]:
import os
import shutil
if os.path.exists("/content/data"):
  print("File exists. Replacing to avoid error.")
  shutil.rmtree("/content/data")

os.mkdir("/content/data")
!wget -P /content/data https://raw.githubusercontent.com/martinj2-dot/CSC386-Wall-Follower/main/data.txt # Read the downloaded data
data = open("/content/data/data.txt", "r")
print("\nThe file looks like:\n"+data.read())
data.close()

File exists. Replacing to avoid error.
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
--2021-12-03 05:57:11--  https://raw.githubusercontent.com/martinj2-dot/CSC386-Wall-Follower/main/data.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 629 [text/plain]
Saving to: ‘/content/data/data.txt’

data.txt            100%[===================>]     629  --.-KB/s    in 0s      

2021-12-03 05:57:11 (29.4 MB/s) - ‘/content/data/data.txt’ saved [629/629]


The file looks like:
alma
alpena
ann_arbor
benton_harbor
cadillac
detroit
escanaba
flint
grand_rapids
kalamazoo
lansing
mackinaw_city
marquette
saginaw
traverse_city
^ Above are the nodes, below are the distances to the edges
benton_harbor kalama

In [ ]:
class Node: 
  """
  parent: The index of the parent node of the current node
  distances: the distance of each node, inatilized high and updated from data in mi_graph.txt
  visited: boolean type, whether or not the node has been visited
  queued: boolean type, whether or not the node has been queued
  """
  def __init__(self):
    self.parent = -1
    self.distances = 1e6
    self.visited = False
    self.queued = False

class Graph:
  def __init__(self): 
    self.content = open("/content/data/data.txt", "r")
    self.data = []
    self.edges = []
    self.edge_costs = []
    self.createGraph()
  def createGraph(self):
    node = True
    count = 0
    lines = self.content.readlines()
    for line in lines:
      if "^" in line: 
        node = False
        num_nodes = count
      if node: # If the line is a node
        self.data.append(line)
        count +=1
      else:
        for i in range(num_nodes):
          file_to_edge = line.split()
          c1 = nameToIdx(file_to_edge[1], self.data)
          c2 = nameToIdx(file_to_edge[2], self.data)
          self.edges[c1].append(c2)
          self.edges[c2].append(c1)
          self.edge_costs[c1].append(float(file_to_edge[3]))
          self.edge_costs[c2].append(float(file_to_edge[3]))


# Functions we will be using

def nameToIdx(name, v):
  for i in range(len(v)):
    if v[i] == name: return i 
    else: return -1

def tracePath(n, g):
  path = []
  curr = n
  while curr != -1:
    path.append(curr)
    curr = getParent(curr, g)

  # Since we built the path backwards, we need to reverse it.
  reverse(path.begin(), path.end())
  return path

def getNeighbors(n, g): return g.edges[n]

def getEdgeCosts(n, g): return g.edge_costs[n]

def getParent(idx, g): return g.nodes[idx].parent 

def initGraph(g):
  # populates a vector of nodes with size(graph data)
    for i in range(len(g.data)):
      node = Node()
      g.nodes.append(node)

def bfs(start, goal, g):
  initGraph(g)
  path = []
  visit_list = []
  currentIdx = start
  g.nodes[currentIdx].distances = 0
  g.nodes[currentIdx].queued = True
  visit_list.push(currentIdx)

  while not visit_list.empty():
    currentIdx = visit_list[0]
    g.nodes[currentIdx].visited = true
    neighbors = getNeighbors(currentIdx, g)
    edge_weights = getEdgeCosts(currentIdx, g)
    if currentIdx == goal:           # Checks to see if current node is goal node. Returns path of it is
      path = tracePath(currentIdx, g)
      break
    visit_list.pop()

    for i in range(len(neighbors)):                # Populates node data for each node, within the vector of nodes.
      if not g.nodes[neighbors[i]].visited:        # Checks to see if the node has been visited
        if not g.nodes[neighbors[i]].queued:       # Checks to see if the node has been queued
          visit_list.push(neighbors[i])            # Pushes current node to the back of the queue if all conditions are false

        if g.nodes[neighbors[i]].distances > g.nodes[currentIdx].distances + edge_weights[i]: # what happens if we run into a loop
          g.nodes[neighbors[i]].distances = g.nodes[currentIdx].distances + edge_weights[i] # updates distances of current node
          g.nodes[neighbors[i]].parent = currentIdx # sets parent index to current index of the current node
  return path

def main():
  g = Graph()
  start = nameToIdx("ann_arbor", g.data)
  goal = nameToIdx("marquette", g.data)

  f"Searching for a path from {g.data[start]} (index: {start}) to {g.data[goal]} (index: {goal})...\n"
  path = bfs(start, goal, g)
  
  if len(path) < 1:
    print("No path found :(\n")
  else:
    print("Path: ")
    for i in range(len(path)-1): 
      f"{g.data[path[i]]} -> "
    f"(g.data[path.back()]\n"
  return 0

if __name__ == "__main__":
  main()


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-76-5bf3bfe27973>", line 120, in <module>
    main()
  File "<ipython-input-76-5bf3bfe27973>", line 103, in main
    g = Graph()
  File "<ipython-input-76-5bf3bfe27973>", line 20, in __init__
    self.createGraph()
  File "<ipython-input-76-5bf3bfe27973>", line 37, in createGraph
    self.edges[c1].append(c2)
IndexError: list index out of range

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 1823, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'IndexError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File

IndexError: ignored

### Submission Instructions
* Set the link to “Anyone with the link can view”.
* Submit the link to Moodle.